In [1]:
import timm
model = timm.create_model('inception_v4', pretrained=True)
model.eval()

/home/nathanw/miniconda3/envs/env1/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: could not determine the shape of object type 'torch.storage.UntypedStorage'

In [ ]:
import urllib
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
urllib.request.urlretrieve(url, filename)
img = Image.open(filename).convert('RGB')
tensor = transform(img).unsqueeze(0) # transform and add batch dimension

In [ ]:
print(tensor.shape)
print(tensor.dtype)
print(tensor.min())
print(tensor.max())

torch.Size([1, 3, 299, 299])
torch.float32
tensor(-1.)
tensor(1.)


In [ ]:
import torch
with torch.no_grad():
    out = model(tensor)
probabilities = torch.nn.functional.softmax(out[0], dim=0)
print(probabilities.shape)

torch.Size([1000])


In [ ]:
# Get imagenet class mappings
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Print top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())
# prints class names and probabilities like:
# [('Samoyed', 0.6425196528434753), ('Pomeranian', 0.04062102362513542), ('keeshond', 0.03186424449086189), ('white wolf', 0.01739676296710968), ('Eskimo dog', 0.011717947199940681)]

Samoyed 0.9300338625907898
white wolf 0.004212407395243645
Arctic fox 0.002659186953678727
Pomeranian 0.0016048169927671552
keeshond 0.0015494022518396378


In [1]:
from descriptors import *
from configs import open_configs
from load_data import load_data
from format_data import format_data
from trans_recovery import *
import torch

/home/nathanw/miniconda3/envs/env1/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data_configs, training_configs, model_configs = open_configs()
# print('configs oppened')
# training_datasets = load_data(data_configs)
# print('data loaded')
# data = torch.utils.data.ConcatDataset(training_datasets)
# print('data concatenated')

# model = Inceptionv4()
# model.eval()

# with torch.no_grad():
#     out = model(data[0][0].unsqueeze(0))
# print(out.shape)
# pyramid = DilationPyramid(model_configs['dilation_pyramid_configs'])
# out = pyramid(out)
# print(out.shape)
data_configs, training_configs, model_configs = open_configs()
print('configs oppened')
training_datasets = load_data(data_configs)
print('data loaded')
data = torch.utils.data.ConcatDataset(training_datasets)
print('data concatenated')

trans_recovery_configs = model_configs['trans_recovery_configs']
dilation_pyramid_configs = model_configs['dilation_pyramid_configs']
model = TranRecovery(dilation_pyrimand_configs=dilation_pyramid_configs, data_configs=data_configs, trans_recovery_configs=trans_recovery_configs)
model.eval()

with torch.no_grad():
    out = model(data[0][0].unsqueeze(0))
print(out.shape)


configs oppened
50000
Loading 100 took 2.0001678466796875 seconds
data loaded
data concatenated
S1_x shape:  torch.Size([1, 64, 29, 29])
S1_x0 shape:  torch.Size([1, 64, 27, 27])
S1_x1 shape:  torch.Size([1, 96, 27, 27])
S2_x shape:  torch.Size([1, 192, 25, 25])
S2_x0 shape:  torch.Size([1, 192, 23, 23])
S2_x1 shape:  torch.Size([1, 192, 23, 23])
RA_x0 shape:  torch.Size([1, 384, 21, 21])
RA_x1 shape:  torch.Size([1, 256, 21, 21])
RA_x2 shape:  torch.Size([1, 384, 21, 21])
RB_x0 shape:  torch.Size([1, 192, 19, 19])
RB_x1 shape:  torch.Size([1, 320, 19, 19])
RB_x2 shape:  torch.Size([1, 1024, 19, 19])
S1_x shape:  torch.Size([1, 64, 29, 29])
S1_x0 shape:  torch.Size([1, 64, 27, 27])
S1_x1 shape:  torch.Size([1, 96, 27, 27])
S2_x shape:  torch.Size([1, 192, 25, 25])
S2_x0 shape:  torch.Size([1, 192, 23, 23])
S2_x1 shape:  torch.Size([1, 192, 23, 23])
RA_x0 shape:  torch.Size([1, 384, 21, 21])
RA_x1 shape:  torch.Size([1, 256, 21, 21])
RA_x2 shape:  torch.Size([1, 384, 21, 21])
RB_x0 shap

AttributeError: 'tuple' object has no attribute 'shape'